In [19]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
from netgen.geom2d import SplineGeometry
import numpy as np
from ngsolve.nonlinearsolvers import Newton

We want to solve the PDE-constrained shape optimization problem

$$
\min_{\Omega \subset \mathsf{D}} J(u)
:= \int_\Omega |\nabla u_\Omega - \nabla u_{ref}|^2 \, dx
$$

subject to $(\Omega,u_\Omega)$ satisfying

$$
\int_\Omega \nabla u_\Omega \cdot \nabla v + u_\Omega v \, dx
=
\int_\Omega f v \, dx
\quad \text{for all } v \in H_0^1(\Omega),
$$

where $\Omega \subset \mathbb{R}^2$ and
$u_d, f \in C^1(\mathbb{R}^2)$.


In [20]:

geo = OCCGeometry(Circle((0, 0), 0.9).Face(), dim=2).GenerateMesh(maxh=0.05)
mesh = Mesh(geo)
# #Draw(mesh)

vol = 3e-2

a = 0.6
b = 0.4

ud = 1 - ((x-0.01)**2/a**2 + (y-0.01)**2/b**2)

f = (2/a**2 + 2/b**2) + ud
# # #f = CF(1)


# a =4.0/5.0
# b = 1.8
# f = CoefficientFunction((sqrt((x - a)**2 + b * y**2) - 1) \
#                 * (sqrt((x + a)**2 + b * y**2) - 1) \
#                 * (sqrt(b * x**2 + (y - a)**2) - 1) \
#                 * (sqrt(b * x**2 + (y + a)**2) - 1) - 0.001)

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

#Draw(ud, mesh, "target_function")


In [21]:
fes = H1(mesh, order=2, dirichlet=".*")
u, v = fes.TnT()
gfu = GridFunction(fes)
scene_gfu = Draw (gfu, mesh, "state")

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx
a += u*v*dx

fstate = LinearForm(fes)
fstate += f*v*dx

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [22]:
def SolveStateEquation():
    rhs = gfu.vec.CreateVector()
    rhs.data = fstate.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()) * rhs
    gfu.vec.data += update

In [23]:
a.Assemble()
fstate.Assemble()
SolveStateEquation()
scene_gfu.Redraw()

### Adjoint equation

The adjoint p satisfies

$$
\int_\Omega \nabla w \cdot \nabla p + w p \, dx
=
-2 \int_\Omega (\nabla u_\Omega - \nabla u_{ref}) w 
\quad \text{for all } w \in H_0^1(\Omega),
$$

In [24]:
def Cost(u):
    cost = InnerProduct(grad(u) - grad_ud, grad(u) - grad_ud) * dx
    cost += InnerProduct(CF(vol), CF(1)) * dx

    return cost



p, w = fes.TnT()
gfp = GridFunction(fes)
scene_gfp = Draw (gfp, mesh, "adjoint")

fadjoint = LinearForm(fes)
#fadjoint += -1*Cost(gfu).Diff(gfu,w)
fadjoint += -2* InnerProduct(grad(gfu) - grad_ud, grad(w)) * dx

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [25]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [26]:
fadjoint.Assemble()
SolveAdjointEquation()
scene_gfp.Redraw()

The shape derivative of the shape function

$$
\mathcal{J}(\Omega) := \int_\Omega |\nabla u - \nabla u_d|^2 \, dx
$$

at $\Omega$ in direction $X$ is given by

$$
\begin{aligned}
DJ(\Omega)(X)
&=
\int_\Omega \operatorname{div}(X)|\nabla u - \nabla u_{\mathrm{ref}}|^2
- 2 \int_\Omega (\nabla u - \nabla u_{\mathrm{ref}}) \nabla^2 u_{\mathrm{ref}} \cdot X\\
&\quad
+ \int_\Omega \operatorname{div}(X) u p \\
&\quad
+ \int_\Omega
\bigl(\operatorname{div}(X)I - \partial X - \partial X^\top\bigr)
\nabla u \cdot \nabla p \\
&\quad
- \int_\Omega (\nabla f \cdot X)\,p\\
& \quad
- \int_\Omega \operatorname{div}(X)fp .
\end{aligned}
$$

We now assemble this shape derivative in NGSolve as follows:


In [27]:
def hesse(u):
    return CoefficientFunction(
        (u.Diff(x).Diff(x), u.Diff(x).Diff(y),
         u.Diff(y).Diff(x), u.Diff(y).Diff(y)), 
        dims=(2,2)
    )

#VEC = H1(mesh, order=3, dirichlet=".*", dim=2)
VEC = H1(mesh, order=1, dim = 2)
X, PHI = VEC.TnT()

dJOmega = LinearForm(VEC)
dJOmega += InnerProduct((div(PHI)*(grad(gfu)- grad_ud)), grad(gfu)- grad_ud) * dx
dJOmega += -2* InnerProduct(hesse(ud)*PHI, grad(gfu)- grad_ud) * dx
dJOmega += div(PHI)* gfu *gfp *dx
dJOmega += InnerProduct(div(PHI)*grad(gfu), grad(gfp)) * dx
dJOmega += -InnerProduct(PHI.Deriv()*grad(gfu), grad(gfp)) * dx
dJOmega += -InnerProduct(grad(gfu), PHI.Deriv()*grad(gfp)) * dx
dJOmega += -div(PHI)* f * gfp * dx
dJOmega += -grad_f * PHI*gfp * dx
# added volumen constraint
dJOmega += -vol * div(PHI)* dx 




- Next, we want to find a vector field $X$ which yields a decrease of the objective
  function, i.e. we want to find a vector field $X$ such that

  $$
  d\mathcal{J}(\Omega; X) < 0.
  $$

- This can be achieved by solving an auxiliary boundary value problem of the type

  $$
  \text{Find } X \in H :
  \qquad
  b(X, \Phi) = d\mathcal{J}(\Omega; \Phi)
  \quad \text{for all } \Phi \in H,
  $$

  for a suitable Hilbert space $H$ (e.g. $H = H^1(\Omega)^2$).
  Here, $b(\cdot,\cdot) : H \times H \to \mathbb{R}$ is a positive definite bilinear
  form which we are free to choose. Then $-X$ is a descent direction since

  $$
  d\mathcal{J}(\Omega; -X)
  = - d\mathcal{J}(\Omega; X)
  = - b(X, X)
  < 0.
  $$

- First we start with the bilinear form

  $$
  b(X, Y)
  =
  \int_\Omega \nabla X : \nabla Y + X \cdot Y \, dx .
  $$

- we can also choose X as 
  $$
  \min _{X \in H^1(\Omega)} \frac{1}{p} \int _\Omega |\nabla X|^p - DJ(\Omega)(X)
  $$

  since this is a continous functional on $H^1(\Omega)'$, it is equivalent to finding $X \in H^1(\Omega)$ sucht that, 

  $$
  \int _\Omega |\nabla X | ^{p-2}  \nabla X \cdot \nabla \Phi = DJ(\Omega)(\Phi) \qquad \forall \Phi \in H^1(\Omega)
  $$

In [28]:
def pose_deformation_equation(gamma=None, p = None): 
    b = BilinearForm(VEC)
    # standard case
    if not p:
        b += InnerProduct(grad(X),grad(PHI))*dx + InnerProduct(X,PHI)*dx

    if gamma:
        # standard case with gamma regularization
        print('Using gamma =', gamma)
        b = BilinearForm(VEC)
        b += (1.0/gamma)*InnerProduct(grad(X),grad(PHI))*dx + (1.0/gamma)*InnerProduct(X,PHI)*dx
        #b += InnerProduct(CF((-X[0].Diff(x)+X[1].Diff(y), X[0].Diff(y)+X[1].Diff(x))),
                                   #CF((-PHI[0].Diff(x)+PHI[1].Diff(y), PHI[0].Diff(y)+PHI[1].Diff(x)))) * dx

        b += gamma * (PHI.Deriv()[0,0] - PHI.Deriv()[1,1])*(X.Deriv()[0,0] - X.Deriv()[1,1]) *dx
        b += gamma * (PHI.Deriv()[1,0] + PHI.Deriv()[0,1])*(X.Deriv()[1,0] + X.Deriv()[0,1]) *dx


        #print("expression tree\n")
        #print(b)
        return b
    
    elif p:
        # solving minimization problem
        print('Using p =', p)
        eps = 1E-5
    
        grad_norm = sqrt(InnerProduct(grad(X), grad(X)) + eps**2)
        

        c = BilinearForm(VEC)
        c += (grad_norm**(p-2)) * InnerProduct(grad(X), grad(PHI)) * dx
        c += InnerProduct(X,PHI)*dx
        
        # -DJ(Omega)(PHI)
        c += -InnerProduct((div(PHI)*(grad(gfu)- grad_ud)), grad(gfu)- grad_ud) * dx
        c += 2* InnerProduct(grad(gfu)- grad_ud, hesse(ud)*PHI) * dx
        c += -div(PHI)* gfu *gfp *dx
        c += -InnerProduct(div(PHI)*grad(gfu), grad(gfp)) * dx
        c += InnerProduct(PHI.Deriv()*grad(gfu), grad(gfp)) * dx
        c += InnerProduct(grad(gfu), PHI.Deriv()*grad(gfp)) * dx
        c += div(PHI)* f * gfp * dx
        c += grad_f*PHI*gfp * dx
        c += vol * div(PHI)*dx 
        
        # c += -div(PHI)* f * gfp * dx
        # c += -grad_f*PHI*gfp * dx
        
        return c

        
    #print("expression tree\n")
    #print(b)
    return b
   

def SolveDeformationEquation(linear=True):
    if linear:
        #print("solving linear deformation equation")
        rhs = gfX.vec.CreateVector()
        rhs.data = dJOmega.vec - b.mat * gfX.vec
        update = gfX.vec.CreateVector()
        update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
        gfX.vec.data += update
        #scene_gfset.Redraw()
   
    elif not linear:
        #print("solving nonlinear deformation equation with Newton")
        Newton(c, gfX, maxerr = 1E-8, freedofs=VEC.FreeDofs(), maxit = 1000, printing=True)

        #print("Deformation norm:", Norm(gfX.vec)
        #scene_gfset.Redraw()
        


### checking the mesh quality

The area of a triangle is given by

$$
A = 0.5 a h 
$$

where h is the longest egde and a is the height. 

Now for shape regular finite elements a should be around h, therefore we can investigate

$$
\int_T \frac{2}{h^2} dx \qquad \textbf{elementwise!}
$$

and the (undeformed) mesh, which gives a measure of mesh deformation.


In [44]:
# def check_mesh_quality(mesh, gfSet = None):
#     h = specialcf.mesh_size
#     if gfSet is not None:
#         gftemp = mesh.deformation
#         #gftemp.Set((x,y))

#         jac = Det((grad(gftemp)))
#         # for set function we apply transformation rule since mapping is not stored in Intergate mesh
#         qual = np.array(Integrate(2.0/(h**2) * jac, mesh, element_wise=True))

#     else: 
#         qual = np.array(Integrate(2.0/(h**2), mesh, element_wise=True))

#     qual = (qual)
#     print("=================Mesh Quality Check=================")
#     print("max quality:", np.max(qual), " min quality:", np.min(qual), " mean quality:", np.mean(qual))
#     print("===================================================")
#     #Draw(h, mesh)
    
#     return qual

# #check_mesh_quality(mesh)


In [39]:
def optimize_shape(linear = True, LineSearch = True):
    gfset.Set((0,0))
    mesh.SetDeformation(gfset)
    scene.Redraw()
    a.Assemble()
    fstate.Assemble()
    SolveStateEquation()


    iter_max = 600
    Jold = Integrate(Cost(gfu), mesh)
    converged = False

    # input("Press enter to start optimization")
    print('Cost at initial design', Integrate (Cost(gfu), mesh))
    for k in range(iter_max):
        mesh.SetDeformation(gfset)
        scene.Redraw()

        print('cost at iteration', k, ': ', Jold)

        a.Assemble()
        fstate.Assemble()
        SolveStateEquation()

        fadjoint.Assemble()
        SolveAdjointEquation()
        if linear:
            b.Assemble()
        #plt.spy(b.mat.ToDense())
        dJOmega.Assemble()
        SolveDeformationEquation(linear=linear)

        #scale = 0.01 / Norm(gfX.vec)
        
        scale = 1E-4

        gfsetOld.vec[:] = gfset.vec[:]
        gfset.vec.data -= scale * gfX.vec

        Jnew = Integrate(Cost(gfu), mesh)
        if not LineSearch:
            if Jnew >= Jold:
                print("No descent found for current step size")
                converged = True

        print("gradient norm", np.linalg.norm(dJOmega.vec.FV().NumPy()))
        if LineSearch:
            while Jnew > Jold and scale > 1e-12:
                #input('a')
                scale = scale /2
                scale = min(1E-3, scale)
                print("scale = ", scale)
                #print("scale without normalization factor = ", scale* Norm(gfX.vec))
                if scale <= 1e-12:
                    converged = True
                    break

                gfset.vec.data = gfsetOld.vec - scale * gfX.vec         # scale times -X is descent direction
                mesh.SetDeformation(gfset)                              # applying deformation and solve new state

                a.Assemble()                                           # note that during line search we only have to solve state to check if update yields lower costs              
                fstate.Assemble()
                SolveStateEquation()
                Jnew = Integrate(Cost(gfu), mesh)
                print("New cost during line search: ", Jnew)
        
        #print("scaling parameter:", scale)
        #print("descet direction value", -1*np.dot(dJOmega.vec.FV().NumPy(), gfX.vec.FV().NumPy()))

        if converged==True:
            print("No more descent can be found")
            break
        Jold = Jnew

        Redraw(blocking=True)

    print("Norm of gset", Integrate(gfset**2, mesh))

### linear

In [45]:
#check_mesh_quality(mesh)
gfX = GridFunction(VEC)
b = pose_deformation_equation()

gfset = GridFunction(VEC)
gfset.Set((0,0))
gfsetOld = GridFunction(VEC)

scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)



optimize_shape(linear = True)
mesh.SetDeformation(gfset)
#check_mesh_quality(mesh, gfset)

# gfX2 = GridFunction(VEC)
# gfX2.vec.data =
# print("=====")
# print(np.dot(dJOmega.vec.FV().NumPy(), gfX.vec.FV().NumPy()))
# print(Integrate(Cost(gfu), mesh))


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Cost at initial design 13.962313603874364
cost at iteration 0 :  13.962313603874364
gradient norm 4.378716560509942
cost at iteration 1 :  13.735479642407437
gradient norm 4.347433700275001
cost at iteration 2 :  13.526573317812495
gradient norm 4.317982493312136
cost at iteration 3 :  13.322324423530375
gradient norm 4.290286870183119
cost at iteration 4 :  13.122591983622375
gradient norm 4.264273392268938
cost at iteration 5 :  12.927240619761761
gradient norm 4.239871172220026
cost at iteration 6 :  12.736140272800176
gradient norm 4.217011799140467
cost at iteration 7 :  12.54916594090012
gradient norm 4.195629267803965
cost at iteration 8 :  12.366197433099561
gradient norm 4.175659911284962
cost at iteration 9 :  12.187119137265825
gradient norm 4.1570423364647136
cost at iteration 10 :  12.011819801468183
gradient norm 4.139717361947628
cost at iteration 11 :  11.840192327878066
gradient norm 4.1236279579899
cost at iteration 12 :  11.672133578365985
gradient norm 4.10871918810

KeyboardInterrupt: 

In [ ]:
#check_mesh_quality(mesh)
gfX = GridFunction(VEC)

b = pose_deformation_equation(gamma=1)

gfset = GridFunction(VEC)
gfset.Set((0,0))
gfsetOld = GridFunction(VEC)

scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)



optimize_shape(linear = True)
print("Mesh quality after optimization (min, max): ")
mesh.SetDeformation(gfset)
#_ = check_mesh_quality(mesh, gfSet=gfset)

# gfX2 = GridFunction(VEC)
# gfX2.vec.data =
# print("=====")
# print(np.dot(dJOmega.vec.FV().NumPy(), gfX.vec.FV().NumPy()))
# print(Integrate(Cost(gfu), mesh))


=================Mesh Quality Check=================
max quality: 0.9999999999999962  min quality: 0.9999999999999957  mean quality: 0.999999999999996
Using gamma = 1


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Cost at initial design 13.962313603874366
cost at iteration 0 :  13.962313603874366
gradient norm 4.378716560509939
cost at iteration 1 :  13.825699181990547
gradient norm 4.355986771584864
cost at iteration 2 :  13.708328365784794
gradient norm 4.333802657575129
cost at iteration 3 :  13.592661771235601
gradient norm 4.312146668568248
cost at iteration 4 :  13.47865822454423
gradient norm 4.291002080926099
cost at iteration 5 :  13.366278181855678
gradient norm 4.2703529463614025
cost at iteration 6 :  13.255483638371954
gradient norm 4.250184044537727
cost at iteration 7 :  13.14623804349612
gradient norm 4.230480838941728
cost at iteration 8 :  13.038506221570902
gradient norm 4.211229435795784
cost at iteration 9 :  12.932254297811912
gradient norm 4.19241654579348
cost at iteration 10 :  12.827449629064052
gradient norm 4.174029448457088
cost at iteration 11 :  12.724060739036213
gradient norm 4.156055958929718
cost at iteration 12 :  12.622057257697548
gradient norm 4.13848439702

### non linear with p

In [ ]:
#_ = check_mesh_quality(mesh)
c = pose_deformation_equation(p=2)

gfX = GridFunction(VEC)
gfX.Set((x *(1-x)*y*(1-y),0))

gfsetOld = GridFunction(VEC)
gfset = GridFunction(VEC)
gfset.Set((0,0))

scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)


optimize_shape(linear=False)
mesh.SetDeformation(gfset)
#_ = check_mesh_quality(mesh, gfSet=gfset)

=================Mesh Quality Check=================
max quality: 0.9999999999999962  min quality: 0.9999999999999957  mean quality: 0.999999999999996
Using p = 2


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Cost at initial design 13.962313603874358
cost at iteration 0 :  13.962313603874358
Newton iteration  0
err =  37.41125423020095
Newton iteration  1
err =  1.367183732002221e-13
gradient norm 4.37871656050994
cost at iteration 1 :  13.735535077132992
Newton iteration  0
err =  0.5046100964236618
Newton iteration  1
err =  4.6608971078953086e-14
gradient norm 4.347445803682443
cost at iteration 2 :  13.526663193302609
Newton iteration  0
err =  0.49390339718914034
Newton iteration  1
err =  4.440364840398551e-14
gradient norm 4.318006503005602
cost at iteration 3 :  13.322447146192985
Newton iteration  0
err =  0.48353466560852787
Newton iteration  1
err =  4.391404966389322e-14
gradient norm 4.290322609532709
cost at iteration 4 :  13.122746031561732
Newton iteration  0
err =  0.473490108635273
Newton iteration  1
err =  4.204956224256386e-14
gradient norm 4.264320703513839
cost at iteration 5 :  12.927424539251607
Newton iteration  0
err =  0.46375659269726616
Newton iteration  1
err 

In [ ]:
#check_mesh_quality(mesh)
c = pose_deformation_equation(p=5)

gfX = GridFunction(VEC)
gfX.Set((x *(1-x)*y*(1-y),0))

gfset = GridFunction(VEC)
gfsetOld = GridFunction(VEC)
gfset.Set((0,0))

scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

optimize_shape(linear=False)
mesh.SetDeformation(gfset)
#_ = check_mesh_quality(mesh, gfSet=gfset)

=================Mesh Quality Check=================
max quality: 0.9999999999999962  min quality: 0.9999999999999957  mean quality: 0.999999999999996
Using p = 5


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Cost at initial design 13.962313603874366
cost at iteration 0 :  13.962313603874366
Newton iteration  0
err =  69.41085956977342
Newton iteration  1
err =  118002684.64594492
Newton iteration  2
err =  57483743.97270619
Newton iteration  3
err =  28002590.20405723
Newton iteration  4
err =  13641161.899756689
Newton iteration  5
err =  6645145.917537667
Newton iteration  6
err =  3237111.661716003
Newton iteration  7
err =  1576924.2752873693
Newton iteration  8
err =  768181.7711847737
Newton iteration  9
err =  374211.5222027478
Newton iteration  10
err =  182293.13492178012
Newton iteration  11
err =  88802.14797248352
Newton iteration  12
err =  43259.0148611245
Newton iteration  13
err =  21073.165221200088
Newton iteration  14
err =  10265.566167143166
Newton iteration  15
err =  5000.760198686759
Newton iteration  16
err =  2436.0665100807973
Newton iteration  17
err =  1186.7034222145114
Newton iteration  18
err =  578.0895780395631
Newton iteration  19
err =  281.6099054501481